# **Análisis Exploratorio de Datos de la Fórmula 1**
_Egoitz Aulestia Padilla — Oct 2024_

## Librerías
Importamos las librerías necesariaas


In [1]:
import pandas as pd

## Carga de Datasets
Cargamos todos los datasets

In [2]:
f1_circuits = pd.read_csv('/content/sample_data/circuits.csv')
f1_constructor_results = pd.read_csv('/content/sample_data/constructor_results.csv')
f1_constructor_standings = pd.read_csv('/content/sample_data/constructor_standings.csv')
f1_constructors = pd.read_csv('/content/sample_data/constructors.csv')
f1_driver_standings = pd.read_csv('/content/sample_data/driver_standings.csv')
f1_drivers = pd.read_csv('/content/sample_data/drivers.csv')
f1_lap_times = pd.read_csv('/content/sample_data/lap_times.csv')
f1_pit_stops = pd.read_csv('/content/sample_data/pit_stops.csv')
f1_qualifying = pd.read_csv('/content/sample_data/qualifying.csv')
f1_races = pd.read_csv('/content/sample_data/races.csv')
f1_results = pd.read_csv('/content/sample_data/results.csv')
f1_seasons = pd.read_csv('/content/sample_data/seasons.csv')
f1_sprint_results = pd.read_csv('/content/sample_data/sprint_results.csv')
f1_status = pd.read_csv('/content/sample_data/status.csv')

---
#### **F1 Circuits** — _Circutos de Carreras_
---

Exploración preliminar de los datos de circuits.csv

Contiene información sobre los circuitos de la F1
- Variables: _circuits.csv_:
  - **circuitId**
  - **circuitRef**
  - **name**
  - **location**
  - **country**
  - **lat** —> _latitud_
  - **lng** —> _longitud_
  - **alt** —> _altitud_
  - **url**

- _Datos de interés a extraer_:
  - Localización geográfica de los circuitos _(cartográfica)_
  - Distancia de KM entre circuitos
  - Nº de circuitos por país
  - Nº de circuitos por continente _(Dict(Continente: List(Paises)))_
  - Altura situada de cada circuito


In [3]:
f1_circuits.head(3)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.7380,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_Internati...


In [4]:
# prompt: muéstrame el circuito de barcelona en el mapa. Quiero ver la trazada de la carretera. Por lo que tendras que hacer zoom. corrige el código para que el mapa se muestre en un rectangulo un menos alargado. Haz mucho zoom

import folium

# Filtrar el circuito de Barcelona
barcelona_circuit = f1_circuits[f1_circuits['circuitRef'] == 'catalunya']

# Obtener la latitud y longitud
latitude = barcelona_circuit['lat'].values[0]
longitude = barcelona_circuit['lng'].values[0]

# Crear el mapa centrado en el circuito de Barcelona
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=15)

# Añadir un marcador al circuito de Barcelona
folium.Marker(
    location=[latitude, longitude],
    popup="Circuit de Barcelona-Catalunya",
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(map_barcelona)


# Ajustar el tamaño del mapa
map_barcelona.fit_bounds([[latitude - 0.02, longitude - 0.02], [latitude + 0.02, longitude + 0.02]])

# Mostrar el mapa
map_barcelona



In [5]:
# prompt: mida la distacia entre dos circuitos en kilometros

from geopy.distance import geodesic

def calculate_circuit_distance(circuit1_lat, circuit1_lng, circuit2_lat, circuit2_lng):
  """
  Calcula la distancia entre dos circuitos en kilómetros utilizando sus coordenadas.

  Args:
    circuit1_lat: Latitud del primer circuito.
    circuit1_lng: Longitud del primer circuito.
    circuit2_lat: Latitud del segundo circuito.
    circuit2_lng: Longitud del segundo circuito.

  Returns:
    La distancia entre los dos circuitos en kilómetros.
  """
  try:
    coord1 = (circuit1_lat, circuit1_lng)
    coord2 = (circuit2_lat, circuit2_lng)
    distance = geodesic(coord1, coord2).kilometers
    return distance
  except:
    return None


# Ejemplo de uso:
circuit1_row = f1_circuits.iloc[0]  # Primer circuito
circuit2_row = f1_circuits.iloc[1]  # Segundo circuito

distance_km = calculate_circuit_distance(circuit1_row['lat'], circuit1_row['lng'],
                                      circuit2_row['lat'], circuit2_row['lng'])

if distance_km:
  print(f"La distancia entre {circuit1_row['name']} y {circuit2_row['name']} es de {distance_km:.2f} km.")
else:
  print("No se pudo calcular la distancia.")


La distancia entre Albert Park Grand Prix Circuit y Sepang International Circuit es de 6325.33 km.


In [6]:
# prompt: traza una linea en un mapa entre el primer circuito y el segundo circuito y pon el número de kilometros

import folium

# Crea un mapa centrado en la ubicación promedio de los dos circuitos
map_center = [(circuit1_row['lat'] + circuit2_row['lat']) / 2,
              (circuit1_row['lng'] + circuit2_row['lng']) / 2]
my_map = folium.Map(location=map_center, zoom_start=3)

# Agrega marcadores para los dos circuitos
folium.Marker(location=[circuit1_row['lat'], circuit1_row['lng']],
              popup=circuit1_row['name']).add_to(my_map)
folium.Marker(location=[circuit2_row['lat'], circuit2_row['lng']],
              popup=circuit2_row['name']).add_to(my_map)


# Dibuja una línea entre los dos circuitos
folium.PolyLine(locations=[[circuit1_row['lat'], circuit1_row['lng']],
                           [circuit2_row['lat'], circuit2_row['lng']]],
                color='blue',
                weight=2.5,
                opacity=1).add_to(my_map)

# Agrega un texto con la distancia entre los circuitos
folium.Marker(location=[(circuit1_row['lat'] + circuit2_row['lat']) / 2,
                        (circuit1_row['lng'] + circuit2_row['lng']) / 2],
              popup=f"Distancia: {distance_km:.2f} km",
              icon=folium.Icon(color='red')).add_to(my_map)

# Muestra el mapa
my_map


In [7]:
# prompt: puedes trazar ahora la linea entre 4 circuitos

# Lista de circuitos a conectar
circuit_indices = [0, 1, 2, 3, 4, 5]  # Puedes cambiar estos índices para elegir otros circuitos

# Crea un mapa centrado en la ubicación promedio de los circuitos
circuit_locations = []
for index in circuit_indices:
  circuit_row = f1_circuits.iloc[index]
  circuit_locations.append([circuit_row['lat'], circuit_row['lng']])

map_center = [(sum(lat for lat, lng in circuit_locations)) / len(circuit_locations),
              (sum(lng for lat, lng in circuit_locations)) / len(circuit_locations)]
my_map = folium.Map(location=map_center, zoom_start=3)

# Agrega marcadores para los circuitos
for index in circuit_indices:
  circuit_row = f1_circuits.iloc[index]
  folium.Marker(location=[circuit_row['lat'], circuit_row['lng']],
                popup=circuit_row['name']).add_to(my_map)


# Dibuja líneas entre los circuitos
for i in range(len(circuit_indices) - 1):
  circuit1_row = f1_circuits.iloc[circuit_indices[i]]
  circuit2_row = f1_circuits.iloc[circuit_indices[i+1]]

  distance_km = calculate_circuit_distance(circuit1_row['lat'], circuit1_row['lng'],
                                        circuit2_row['lat'], circuit2_row['lng'])

  folium.PolyLine(locations=[[circuit1_row['lat'], circuit1_row['lng']],
                             [circuit2_row['lat'], circuit2_row['lng']]],
                  color='blue',
                  weight=2.5,
                  opacity=1).add_to(my_map)

  # Agrega un texto con la distancia entre los circuitos (opcional)
  folium.Marker(location=[(circuit1_row['lat'] + circuit2_row['lat']) / 2,
                          (circuit1_row['lng'] + circuit2_row['lng']) / 2],
                popup=f"Distancia: {distance_km:.2f} km",
                icon=folium.Icon(color='red')).add_to(my_map)

# Muestra el mapa
my_map


In [8]:
f1_circuits.shape

(77, 9)

In [9]:
f1_circuits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitId   77 non-null     int64  
 1   circuitRef  77 non-null     object 
 2   name        77 non-null     object 
 3   location    77 non-null     object 
 4   country     77 non-null     object 
 5   lat         77 non-null     float64
 6   lng         77 non-null     float64
 7   alt         77 non-null     int64  
 8   url         77 non-null     object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.5+ KB


In [10]:
f1_circuits.head(3)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.7380,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.5106,7,http://en.wikipedia.org/wiki/Bahrain_Internati...


In [11]:
circuits_descripcion = f1_circuits.drop(columns=["circuitId", "url"]).describe(include="all").T
circuits_descripcion["Tipos"] = f1_circuits.dtypes
circuits_descripcion["Valores_únicos"] = f1_circuits.nunique()
circuits_descripcion["Valores_Nulos"] = f1_circuits.isnull().sum()
circuits_descripcion["Valores_Nulos_%"] = circuits_descripcion["Valores_Nulos"] / f1_circuits.shape[0] * 100
circuits_descripcion

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Tipos,Valores_únicos,Valores_Nulos,Valores_Nulos_%
circuitRef,77,77,albert_park,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,77,0,0.0
name,77,77,Albert Park Grand Prix Circuit,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,77,0,0.0
location,77,75,Barcelona,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,75,0,0.0
country,77,35,USA,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,35,0,0.0
lat,77.0,NaN,NaN,NaN,33.442925,22.808866,-37.8497,32.7774,40.9517,46.9589,57.2653,float64,77,0,0.0
lng,77.0,NaN,NaN,NaN,1.076683,65.516951,-118.189,-9.39417,3.93083,19.2486,144.968,float64,77,0,0.0
alt,77.0,NaN,NaN,NaN,247.012987,362.738469,-7.0,18.0,129.0,332.0,2227.0,int64,66,0,0.0


.

---
#### **F1 Constructor Results** — _Resultados de los Constructores_
---

Exploración preliminar de los datos de constructor_results.csv

Contiene información sobre los resultados de los constructores en las carreras de F1
- Variables: _constructor_results.csv_:
  - **constructorResultsId**
  - **raceId**
  - **constructorId**
  - **points**
  - **status**

- _Datos de interés a extraer_:
  - Puntos conseguidos por cada constructor
  - Carreras finalizas por cada constructor
  - Carreras accidentadas por cada constructor
  - Lo mismo para pilotos mediante results.csv


In [12]:
f1_constructor_results.head(5)

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,\N
1,2,18,2,8.0,\N
2,3,18,3,9.0,\N
3,4,18,4,5.0,\N
4,5,18,5,2.0,\N


In [13]:
constructor_results_descripcion = f1_constructor_results.drop(columns=["constructorResultsId"]).describe(include="all").T
constructor_results_descripcion["Tipos"] = f1_constructor_results.dtypes
constructor_results_descripcion["Valores_únicos"] = f1_constructor_results.nunique()
constructor_results_descripcion["Valores_Nulos"] = f1_constructor_results.isnull().sum()
constructor_results_descripcion["Valores_Nulos_%"] = constructor_results_descripcion["Valores_Nulos"] / f1_constructor_results.shape[0] * 100
constructor_results_descripcion

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Tipos,Valores_únicos,Valores_Nulos,Valores_Nulos_%
raceId,12505.0,NaN,NaN,NaN,522.379608,310.543824,1.0,285.0,484.0,739.0,1132.0,int64,1048,0,0.0
constructorId,12505.0,NaN,NaN,NaN,45.513315,58.912037,1.0,6.0,22.0,53.0,215.0,int64,175,0,0.0
points,12505.0,NaN,NaN,NaN,3.986166,7.759519,0.0,0.0,0.0,4.0,66.0,float64,60,0,0.0
status,12505,2,\N,12488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,2,0,0.0


In [14]:
f1_constructor_results.shape

(12505, 5)

In [15]:
f1_constructor_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12505 entries, 0 to 12504
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   constructorResultsId  12505 non-null  int64  
 1   raceId                12505 non-null  int64  
 2   constructorId         12505 non-null  int64  
 3   points                12505 non-null  float64
 4   status                12505 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 488.6+ KB


##### Puntos conseguidos por cada constructor

In [16]:
# prompt: muestra los puntos conseguidos por cada constructor

import pandas as pd
# Agrupa por constructorId y suma los puntos
puntos_por_constructor = f1_constructor_results.groupby('constructorId')['points'].sum()

# Une con la tabla de constructores para obtener el nombre del constructor
puntos_por_constructor = pd.merge(puntos_por_constructor, f1_constructors[['constructorId', 'name']], on='constructorId')

# Ordena por puntos de mayor a menor
puntos_por_constructor = puntos_por_constructor.sort_values('points', ascending=False)

print("Puntos conseguidos por cada constructor:")
puntos_por_constructor.head(10)


Puntos conseguidos por cada constructor:


,constructorId,points,name
5,6,10001.0,Ferrari
8,9,7621.0,Red Bull
124,131,7443.5,Mercedes
0,1,6685.5,McLaren
2,3,3630.0,Williams
3,4,1777.0,Renault
9,10,1098.0,Force India
31,32,918.0,Team Lotus
21,22,861.5,Benetton
168,208,706.0,Lotus F1


##### Qué es Status?

In [17]:
# prompt: que significa el status en el dataset?

import pandas as pd
# Mostrar las primeras filas del dataset f1_status para entender el significado de "status"
print(f1_status.head(10))

# Contar la frecuencia de cada valor de status en el dataset f1_results
status_counts = f1_results['statusId'].value_counts()
print(status_counts)

# Combinar el dataset f1_results con f1_status para obtener el significado de statusId
results_with_status = pd.merge(f1_results, f1_status, left_on='statusId', right_on='statusId', how='left')
print(results_with_status[['resultId', 'status']].head(10))

   statusId        status
0         1      Finished
1         2  Disqualified
2         3      Accident
3         4     Collision
4         5        Engine
5         6       Gearbox
6         7  Transmission
7         8        Clutch
8         9    Hydraulics
9        10    Electrical
statusId
1      7528
11     3975
5      2023
12     1611
3      1059
       ... 
72        1
59        1
58        1
102       1
87        1
Name: count, Length: 137, dtype: int64
   resultId     status
0         1   Finished
1         2   Finished
2         3   Finished
3         4   Finished
4         5   Finished
5         6     +1 Lap
6         7     Engine
7         8     Engine
8         9  Collision
9        10   Accident


.

---
#### **F1 Constructor Standings** — Puestos de los Constructores_
---

Exploración preliminar de los datos de constructor_standings.csv

Contiene información sobre los uestos de los constructores en las carreras de F1
- Variables: _constructor_standings.csv_:
  - **constructorStandingsId**
  - **raceId**
  - **constructorId**
  - **points**
  - **position**
  - **positionText**
  - **wins**

- _Datos de interés a extraer_:
  -


In [18]:
f1_constructor_standings.head(5)

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
2,3,18,3,9.0,2,2,0
3,4,18,4,5.0,4,4,0
4,5,18,5,2.0,5,5,0


In [19]:
constructor_standings_descripcion = f1_constructor_standings.drop(columns=["constructorStandingsId"]).describe(include="all").T
constructor_standings_descripcion["Tipos"] = f1_constructor_standings.dtypes
constructor_standings_descripcion["Valores_únicos"] = f1_constructor_standings.nunique()
constructor_standings_descripcion["Valores_Nulos"] = f1_constructor_standings.isnull().sum()
constructor_standings_descripcion["Valores_Nulos_%"] = constructor_standings_descripcion["Valores_Nulos"] / f1_constructor_standings.shape[0] * 100
constructor_standings_descripcion

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Tipos,Valores_únicos,Valores_Nulos,Valores_Nulos_%
raceId,13271.0,NaN,NaN,NaN,529.946425,303.677642,1.0,300.0,504.0,733.0,1132.0,int64,1049,0,0.0
constructorId,13271.0,NaN,NaN,NaN,49.219124,60.744968,1.0,6.0,25.0,58.0,215.0,int64,160,0,0.0
points,13271.0,NaN,NaN,NaN,35.681072,80.56103,0.0,0.0,7.0,32.0,860.0,float64,557,0,0.0
position,13271.0,NaN,NaN,NaN,7.241655,4.363927,1.0,4.0,7.0,10.0,22.0,int64,22,0,0.0
positionText,13271,23,1,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,23,0,0.0
wins,13271.0,NaN,NaN,NaN,0.686987,1.868708,0.0,0.0,0.0,0.0,21.0,int64,22,0,0.0


In [20]:
f1_constructor_standings.shape

(13271, 7)

In [21]:
f1_constructor_standings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13271 entries, 0 to 13270
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   constructorStandingsId  13271 non-null  int64  
 1   raceId                  13271 non-null  int64  
 2   constructorId           13271 non-null  int64  
 3   points                  13271 non-null  float64
 4   position                13271 non-null  int64  
 5   positionText            13271 non-null  object 
 6   wins                    13271 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 725.9+ KB


---
#### **F1 Constructors** — _Constructores_
---

Exploración preliminar de los datos de constructors.csv

Contiene información sobre los puestos de los constructores en las carreras de F1
- Variables: _constructors.csv_:
  - **constructorId**
  - **constructorRef**
  - **name**
  - **nationality**
  - **url**

- _Datos de interés a extraer_:
  - Nombres de los constructores
  - Nacionalidad de cada constructor
  - Número de constructores por países
  - Número de construcoters por continente


In [22]:
f1_constructors.head(3)

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...


In [23]:
constructors_descripcion = f1_constructors.drop(columns=["constructorId", "url"]).describe(include="all").T
constructors_descripcion["Tipos"] = f1_constructors.dtypes
constructors_descripcion["Valores_únicos"] = f1_constructors.nunique()
constructors_descripcion["Valores_Nulos"] = f1_constructors.isnull().sum()
constructors_descripcion["Valores_Nulos_%"] = constructors_descripcion["Valores_Nulos"] / f1_constructors.shape[0] * 100
constructors_descripcion

,count,unique,top,freq,Tipos,Valores_únicos,Valores_Nulos,Valores_Nulos_%
constructorRef,212,212,mclaren,1,object,212,0,0.0
name,212,212,McLaren,1,object,212,0,0.0
nationality,212,24,British,86,object,24,0,0.0


In [24]:
f1_constructors.shape

(212, 5)

In [25]:
f1_constructors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   constructorId   212 non-null    int64 
 1   constructorRef  212 non-null    object
 2   name            212 non-null    object
 3   nationality     212 non-null    object
 4   url             212 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.4+ KB


---
#### **F1 Driver Standings** — _Puestos de los Pilotos_
---

Exploración preliminar de los datos de driver_standings.csv

Contiene información sobre los puestos de los pilots en las carreras de F1
- Variables: _driver_standings.csv_:
  - **driverStandingsId**
  - **raceId**
  - **driverId**
  - **points**
  - **position**
  - **positionText**
  - **wins**

- _Datos de interés a extraer_:
  - Piloto con más victorias
  - Top 10 pilotos con más victo
  - Punots conseguidos por los pilotos
  - Pilots con más punots


In [26]:
f1_driver_standings.head(3)

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0


In [38]:
driver_standings_descripcion = f1_driver_standings.drop(columns=["driverStandingsId"]).describe(include="all").T
driver_standings_descripcion["Tipos"] = f1_driver_standings.dtypes
driver_standings_descripcion["Valores_únicos"] = f1_driver_standings.nunique()
driver_standings_descripcion["Valores_Nulos"] = f1_driver_standings.isnull().sum()
driver_standings_descripcion["Valores_Nulos_%"] = driver_standings_descripcion["Valores_Nulos"] / f1_driver_standings.shape[0] * 100
driver_standings_descripcion

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Tipos,Valores_únicos,Valores_Nulos,Valores_Nulos_%
raceId,34595.0,NaN,NaN,NaN,580.120104,289.290183,1.0,352.0,600.0,803.0,1132.0,int64,1113,0,0.0
driverId,34595.0,NaN,NaN,NaN,313.46076,272.052171,1.0,87.0,222.0,517.0,860.0,int64,852,0,0.0
points,34595.0,NaN,NaN,NaN,14.114939,37.235186,0.0,0.0,1.0,10.0,575.0,float64,429,0,0.0
position,34595.0,NaN,NaN,NaN,19.778928,16.331054,1.0,8.0,16.0,26.0,108.0,int64,108,0,0.0
positionText,34595,109,1,1113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,109,0,0.0
wins,34595.0,NaN,NaN,NaN,0.273074,1.02452,0.0,0.0,0.0,0.0,19.0,int64,20,0,0.0


In [29]:
f1_driver_standings.shape

(34595, 7)

In [30]:
f1_driver_standings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34595 entries, 0 to 34594
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   driverStandingsId  34595 non-null  int64  
 1   raceId             34595 non-null  int64  
 2   driverId           34595 non-null  int64  
 3   points             34595 non-null  float64
 4   position           34595 non-null  int64  
 5   positionText       34595 non-null  object 
 6   wins               34595 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 1.8+ MB


---
---
#### **F1 Drivers** — _Pilotos_
---

Exploración preliminar de los datos de drivers.csv

Contiene información sobre los pilots de carreras de F1
- Variables: _drivers.csv_:
  - **driverId**
  - **driverRef**
  - **number**
  - **code**
  - **forename**
  - **dob**
  - **nationality**
  - **url**

- _Datos de interés a extraer_:
  - Nacionalidad de los pilotos
  - Años de nacimiento de los pilotos
  - Astrología: Horoscopo de los pilotos
  - Horoscopo con mayores victorias
  - Nación con más pilotos
  - Continente con más pilotos
  - Nacioneces con pilotos con más victorias
  -


In [31]:
f1_drivers.head(3)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg


In [41]:
drivers_descripcion = f1_drivers.describe(include="all").T
drivers_descripcion["Tipos"] = f1_drivers.dtypes
drivers_descripcion["Valores_únicos"] = f1_drivers.nunique()
drivers_descripcion["Valores_Nulos"] = f1_drivers.isnull().sum()
drivers_descripcion["Valores_Nulos_%"] = drivers_descripcion["Valores_Nulos"] / f1_drivers.shape[0] * 100
drivers_descripcion

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Tipos,Valores_únicos,Valores_Nulos,Valores_Nulos_%
driverId,859.0,NaN,NaN,NaN,430.059371,248.213115,1.0,215.5,430.0,644.5,860.0,int64,859,0,0.0
driverRef,859,859,hamilton,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,859,0,0.0
number,859,47,\N,802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,47,0,0.0
code,859,97,\N,757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,97,0,0.0
forename,859,478,John,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,478,0,0.0
surname,859,800,Taylor,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,800,0,0.0
dob,859,841,1918-10-06,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,841,0,0.0
nationality,859,42,British,166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,42,0,0.0
url,859,859,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,859,0,0.0


In [33]:
f1_drivers.shape

(859, 9)

In [34]:
f1_drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   driverId     859 non-null    int64 
 1   driverRef    859 non-null    object
 2   number       859 non-null    object
 3   code         859 non-null    object
 4   forename     859 non-null    object
 5   surname      859 non-null    object
 6   dob          859 non-null    object
 7   nationality  859 non-null    object
 8   url          859 non-null    object
dtypes: int64(1), object(8)
memory usage: 60.5+ KB


---
---
---
#### **F1 Lap Times** — _Tiempo de Vueltas_
---

Exploración preliminar de los datos de lap_times.csv

Contiene información sobre los puestos de los pilots en las carreras de F1
- Variables: _lap_times.csv_:
  - **raceId**
  - **driverId**
  - **lap**
  - **position**
  - **time**
  - **milliseconds**

- _Datos de interés a extraer_:
  - _* (para cáculos de tiempo usar milliseconds?)_
  - Vuleta más rapida de cada circuito
  - Evolución vuelta más rapida a lo largo del tiempo (Cada vez más rápidas?)
  - Duración de cada carrera
  - Evolución de duración de cada carrera (Cada vez más rápidas?)
  - Tiempo total en pista por piloto
  - Posición de los pilotos por carrera
  -

In [35]:
f1_lap_times.head(3)

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713


In [40]:
lap_time_descripcion = f1_lap_times.describe(include="all").T
lap_time_descripcion["Tipos"] = f1_lap_times.dtypes
lap_time_descripcion["Valores_únicos"] = f1_lap_times.nunique()
lap_time_descripcion["Valores_Nulos"] = f1_lap_times.isnull().sum()
lap_time_descripcion["Valores_Nulos_%"] = lap_time_descripcion["Valores_Nulos"] / f1_lap_times.shape[0] * 100
lap_time_descripcion

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Tipos,Valores_únicos,Valores_Nulos,Valores_Nulos_%
raceId,575029.0,NaN,NaN,NaN,587.413494,431.352273,1.0,137.0,855.0,993.0,1131.0,int64,531,0,0.0
driverId,575029.0,NaN,NaN,NaN,315.316934,384.3391,1.0,15.0,44.0,820.0,860.0,int64,141,0,0.0
lap,575029.0,NaN,NaN,NaN,30.021122,18.417608,1.0,14.0,29.0,44.0,87.0,int64,87,0,0.0
position,575029.0,NaN,NaN,NaN,9.65965,5.531594,1.0,5.0,9.0,14.0,24.0,int64,24,0,0.0
time,575029,75514,1:21.571,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,75514,0,0.0
milliseconds,575029.0,NaN,NaN,NaN,95809.797963,76926.067517,55404.0,81962.0,90627.0,101971.0,7507547.0,int64,75514,0,0.0


In [42]:
f1_lap_times.shape

(575029, 6)

In [43]:
f1_lap_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575029 entries, 0 to 575028
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        575029 non-null  int64 
 1   driverId      575029 non-null  int64 
 2   lap           575029 non-null  int64 
 3   position      575029 non-null  int64 
 4   time          575029 non-null  object
 5   milliseconds  575029 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 26.3+ MB


---
---
---
#### **F1 Lap Times** — _Tiempo de Vueltas_
---

Exploración preliminar de los datos de lap_times.csv

Contiene información sobre los puestos de los pilots en las carreras de F1
- Variables: _lap_times.csv_:
  - **raceId**
  - **driverId**
  - **stop**
  - **lap**
  - **time**
  - **duration**
  -**milliseconds**

- _Datos de interés a extraer_:
  - _* (para cáculos de tiempo usar milliseconds?)_
  - Vuleta más rapida de cada circuito
  - Evolución vuelta más rapida a lo largo del tiempo (Cada vez más rápidas?)
  - Duración de cada carrera
  - Evolución de duración de cada carrera (Cada vez más rápidas?)
  - Tiempo total en pista por piloto
  - Posición de los pilotos por carrera
  -

In [48]:
f1_pit_stops.head(3)

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426


In [ ]:
lap_time_descripcion = f1_lap_times.describe(include="all").T
lap_time_descripcion["Tipos"] = f1_lap_times.dtypes
lap_time_descripcion["Valores_únicos"] = f1_lap_times.nunique()
lap_time_descripcion["Valores_Nulos"] = f1_lap_times.isnull().sum()
lap_time_descripcion["Valores_Nulos_%"] = lap_time_descripcion["Valores_Nulos"] / f1_lap_times.shape[0] * 100
lap_time_descripcion

In [57]:
pit_stops_descripcion = f1_pit_stops.describe(include="all").T
pit_stops_descripcion["Tipos"] = f1_pit_stops.dtypes
pit_stops_descripcion["Valores_únicos"] = f1_pit_stops.nunique()
pit_stops_descripcion["Valores_Nulos"] = f1_pit_stops.isnull().sum()
pit_stops_descripcion["Valores_Nulos_%"] = pit_stops_descripcion["Valores_Nulos"] / f1_pit_stops.shape[0] * 100
pit_stops_descripcion

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,Tipos,Valores_únicos,Valores_Nulos,Valores_Nulos_%
raceId,10990.0,NaN,NaN,NaN,975.731665,89.041843,841.0,893.0,967.0,1055.0,1132.0,int64,273,0,0.0
driverId,10990.0,NaN,NaN,NaN,542.601274,385.555797,1.0,20.0,817.0,832.0,860.0,int64,74,0,0.0
stop,10990.0,NaN,NaN,NaN,1.797179,1.540691,1.0,1.0,2.0,2.0,70.0,int64,14,0,0.0
lap,10990.0,NaN,NaN,NaN,25.314741,14.896984,1.0,13.0,25.0,36.0,78.0,int64,74,0,0.0
time,10990,7983,15:05:16,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,7983,0,0.0
duration,10990,7434,22.534,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,7434,0,0.0
milliseconds,10990.0,NaN,NaN,NaN,85304.309554,311489.432628,12897.0,21951.25,23629.0,26503.5,3069017.0,int64,7434,0,0.0


In [51]:
f1_pit_stops.shape

(10990, 7)

In [52]:
f1_pit_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10990 entries, 0 to 10989
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceId        10990 non-null  int64 
 1   driverId      10990 non-null  int64 
 2   stop          10990 non-null  int64 
 3   lap           10990 non-null  int64 
 4   time          10990 non-null  object
 5   duration      10990 non-null  object
 6   milliseconds  10990 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 601.1+ KB


No se si el siguiente dato teine mucho sentido porque 12 segundos para el pit stop más rápido son demasiados segundos, ya que hoy endia podría rondar entre 1.8 y 3 segundos.

In [54]:
# prompt: pit stop más rapido

# Encuentra el pit stop más rápido
fastest_pit_stop = f1_pit_stops.loc[f1_pit_stops['milliseconds'].idxmin()]

print("Pit stop más rápido:")
print(fastest_pit_stop)


Pit stop más rápido:
raceId               858
driverId             813
stop                   2
lap                   30
time            17:58:49
duration          12.897
milliseconds       12897
Name: 1037, dtype: object


---
---
---
#### **F1 Qualifying** — _Clasificación_
---

Exploración preliminar de los datos de qualifying.csv

Contiene información sobre los puestos de los pilots en las carreras de F1:

Este dataset te permite analizar el rendimiento de los pilotos en las sesiones de clasificación, ver tendencias de tiempos de vuelta, comparar el rendimiento de diferentes pilotos o equipos en diferentes circuitos

- Variables: qualifying.csv_:
  - **qualifyId**
  - **raceId**
  - **driverId**
  - **constructorId**
  - **number**
  - **position**
  - **q1**
  - **q2**
  - **q3**

- _Datos de interés a extraer_:
  - _* (para cáculos de tiempo usar milliseconds?)_
  - En que "Q" se consiguen más pole positions ?
  - Que "Q" suele ser más rápida ?
  - Piloto/s con más pole positions
  - Media de puesto de salida por piloto
  -

In [60]:
f1_qualifying.head(3)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079


In [61]:
f1_qualifying.shape

(10254, 9)

In [62]:
f1_qualifying.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10254 entries, 0 to 10253
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qualifyId      10254 non-null  int64 
 1   raceId         10254 non-null  int64 
 2   driverId       10254 non-null  int64 
 3   constructorId  10254 non-null  int64 
 4   number         10254 non-null  int64 
 5   position       10254 non-null  int64 
 6   q1             10254 non-null  object
 7   q2             10241 non-null  object
 8   q3             10227 non-null  object
dtypes: int64(6), object(3)
memory usage: 721.1+ KB
